In [30]:
# Import dependencies
import pymongo
from pprint import pprint
import pandas as pd
import json

In [31]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)
# assign the uk_food database to a variable name
db = mongo['Project3']
# assign the collection to a variable
collection = db["ship_wreck_data"]

In [20]:
#Setup DB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Project3"]
collection = db["ship_wreck_data"]


In [21]:
#Load data into db
with open("data\TOPO_Shipwrecks_GDA2020.geojson") as file:
    data = json.load(file)
    collection.insert_many(data["features"])

client.close()

# review a document in the establishments collection
pprint(collection.find_one())

{'_id': ObjectId('64be475cb474d686cdc547c1'),
 'geometry': {'coordinates': [139.20135500100002, -36.08200799999997],
              'type': 'Point'},
 'id': 0,
 'properties': {'BUILDDATE': '1838',
                'COUNTRY': 'Australia',
                'FID': 0,
                'HULLDESC': 'Wood',
                'ISFOUND': 'N',
                'ISINSPECTE': 'N',
                'ISPROTECTE': 'Y',
                'JURISDICTI': 'Federal',
                'LATITUDE': -36.082008,
                'LENGTH': 12.2,
                'LONGITUDE': 139.201355,
                'LOSSCAUSE': 'Driven ashore by SW gales.',
                'LOSSDATE': '21/06/1838',
                'LOSSLOCATI': 'Near Cape Bernouilli, 30 miles east of '
                              'Encounter Bay, south of Victor Harbor, South '
                              'Australia.',
                'PORTBUILT': 'Battery Point, Hobart',
                'PUBLIC_ACC': 'General Access',
                'REGION': 'South East',
         

In [22]:
# Update the Country with no meaningful description with Unknown
filter = {"properties.COUNTRY": " "}
newvalues = {"$set": {"properties.COUNTRY":"Unknown"}}
collection.update_many(filter,newvalues)

In [23]:
# Update the Hull Description with no meaningful description with Unknown
filter = {"properties.HULLDESC": " "}
newvalues = {"$set": {"properties.HULLDESC":"Unknown"}}
collection.update_many(filter,newvalues)

In [24]:
# Update the ships with no meaningful description with Ship
filter = {
    "$or": [
        {"properties.RIGDESC": "None"},
        {"properties.RIGDESC": "Unknown"},
        {"properties.RIGDESC": " "}
    ]
}
newvalues = {"$set": {"properties.RIGDESC":"Ship"}}
collection.update_many(filter,newvalues)

In [25]:
# Update the ships with no meaningful description with Ship
filter = {"properties.LOSSDATE":" "}
newvalues = {"$set": {"properties.LOSSDATE":"0000"}}
collection.update_many(filter,newvalues)

In [26]:
# Retrieve all documents in the collection
documents = collection.find()

# Update the LOSSDATE field in each document
for document in documents:
    lossdate = document["properties"]["LOSSDATE"]
    if lossdate:
        document["properties"]["LOSSDATE"] = lossdate[-4:]
        collection.replace_one({"_id": document["_id"]}, document)

In [27]:
# Update the ships with no meaningful description with Ship
filter = {"properties.BUILDDATE":" "}
newvalues = {"$set": {"properties.BUILDDATE":"0000"}}
collection.update_many(filter,newvalues) 


In [28]:
# Retrieve all documents in the collection
documents = collection.find()

# Update the BUILDDATE field in each document
for document in documents:
    builddate = document["properties"]["BUILDDATE"]
    if builddate:
        document["properties"]["BUILDDATE"] = builddate[-4:]
        collection.replace_one({"_id": document["_id"]}, document)

In [29]:
# Change the data type from String to Decimal for BUILDDATE and LOSSDATE
collection.update_many({},[{'$set':{'properties.BUILDDATE': {'$toDouble': '$properties.BUILDDATE'},'properties.LOSSDATE' : {'$toDouble': '$properties.LOSSDATE'}}}])